Basic usage of SAM, taken from: https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb

Before running, make sure you followed the readme. 

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from segment_anything import sam_model_registry,SamPredictor

# Utility functions

### Functions for display purposes, showing masks, points or boxes

In [3]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([255/255, 255/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))  

### Use sam predictor to get masks

In [4]:
def get_masks_from_image(image, predictor, input_point):
    # naïve: return mask from center point
    predictor.set_image(image)
    input_label = np.array([1])

    #Show image with point
    # plt.figure(figsize=(10,10))
    # plt.imshow(image)
    # show_points(input_point, input_label, plt.gca())
    # plt.show()
    
    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    #Show masks
    # for i, (mask, score) in enumerate(zip(masks, scores)):
    #     plt.figure(figsize=(10,10))
    #     plt.imshow(image)
    #     show_mask(mask, plt.gca())
    #     show_points(input_point, input_label, plt.gca())
    #     plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    #     plt.axis('off')
    #     plt.show()  

    return masks


### Get biggest mask in desired style

In: 
* masks: masks generated by sam
* image: opencv image
* output_style: mask, black or white, depending on target model
* mask_type: wide or tight, depending on preference

out: 
numpy image of mask

In [5]:
def get_biggest_mask(masks, image, output_style="mask", mask_style="tight"):
    mask = masks[-1]
    if mask_style == "wide":
        mask = mask.astype(np.uint8)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (100,100))
        dilate = cv2.dilate(mask, kernel, iterations=1)
        mask = np.array(dilate, dtype=bool)

    match output_style:
        case "mask":
            return mask
    
        case "black":
            h, w = mask.shape[-2:]
            masked_image = image * ~mask.reshape(h, w, 1)
            return masked_image
        
        case "white":
            h, w = mask.shape[-2:]
            white_image = np.ones_like(image) * 255
            masked_image = np.where(mask.reshape(h, w, 1), white_image, image)
            return masked_image
        
        case "transparent_white":
            h, w = mask.shape[-2:]
            white_image = np.ones((h,w,4),dtype=np.uint8) * 255
            transparent_image = np.zeros((h,w,4),dtype=np.uint8)
            #transparent_image[:, :, 3] = 0 # Set the alpha channel to 0 (fully transparent) for all pixels
            masked_image = np.where(mask.reshape(h, w, 1), white_image, transparent_image)
            return masked_image

### Example usage

In [6]:

from segment_anything import SamPredictor, sam_model_registry

if False:
    case_nr = "case01" # Specify this!


    input_image_path = "images/"+case_nr+"/"+case_nr+"_input.jpeg"
    mask_path = "images/"+case_nr+"/"+case_nr+"_mask.jpeg"

    sam = sam_model_registry["default"](checkpoint="sam_vit_h_4b8939.pth")
    predictor = SamPredictor(sam)
    print(predictor)
    image = cv2.imread(input_image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    masks = get_masks_from_image(image, predictor)
    wide_mask = get_biggest_mask(masks, image, output_style="transparent_white", mask_style="wide")
    cv2.imwrite("images/masks/truck_wide_mask.png",wide_mask)
    plt.figure(figsize=(10,10))
    plt.imshow(wide_mask)#[:,:,:2])
    plt.show()

    tight_mask = get_biggest_mask(masks, image, output_style="transparent_white", mask_style="tight")
    cv2.imwrite("images/masks/truck_tight_mask.png",tight_mask)
    plt.figure(figsize=(10,10))
    plt.imshow(tight_mask)
    plt.show()

### All in one function to be called from other files

In [7]:
def sam_generate_mask(image_path, mask_style="wide", output_style="transparent_white",save_as_file=False,mask_path=""):
    sam = sam_model_registry["default"](checkpoint="sam_vit_h_4b8939.pth")
    predictor = SamPredictor(sam)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    input_point = np.array([[-1, -1]])
    def click(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            input_point = np.array([[x, y]])
            param[0] = input_point
            print(f"Point selected: {input_point}")
        
    cv2.imshow("image",image)
    cv2.setMouseCallback("image", click, input_point)
    while (input_point == np.array([[-1, -1]])).all():
        cv2.waitKey(10)    
    print("Point selected")
    cv2.destroyAllWindows()
    
    masks = get_masks_from_image(image, predictor, input_point)
    mask = get_biggest_mask(masks, image, output_style=output_style, mask_style=mask_style)
    if save_as_file and mask_path!="":
        cv2.imwrite(mask_path,mask)
    return mask



In [18]:
case_nr = "case10" # Specify this!
mask_type = "wide" # and this



input_image_path = "images/"+case_nr+"/"+case_nr+"_input.png"
mask_path = "images/"+case_nr+"/"+case_nr+"_"+mask_type+"_gan_mask.png"

sam_generate_mask(input_image_path,mask_style=mask_type,output_style="transparent_white",save_as_file=True,mask_path=mask_path)

Point selected: [[355 396]]
Point selected


array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       ...,

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        ...,
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=uint8)

### Turn jupyter notebook into script
if you want to use it in other files
```
from sam_generate_masks import sam_generate_mask
```

In [9]:
#!jupyter nbconvert --to script sam_generate_masks.ipynb